<a href="https://colab.research.google.com/github/liuy01510/portfolio/blob/master/Python/Project-Euler/Project_Euler_Problem_69_(PF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
- Requires the calculation of the relative primes of each number (n) for n $\leq 10^6$.
- Performing brute-force method would likely take too long to solve.
- Requires the use of the general formula.

## General formula
Let $\phi (n)$ be the number of primes that are relative to n.

Therefore, $\phi (n)$ can be determined with the following formula:

Given that $n=\prod_{i=1}^{j}p_i^{a_i}$ where $p_i$ is a prime factor of $n$,

\begin{equation}
\phi (n) = n \cdot \prod_{i=1}^{j} \left(1-\frac{1}{p_i} \right)
\end{equation}

# Solution Procedure
1. For each n, determine their corresponding prime factors.
2. Substitute their prime factors into the general formula given to determine $\phi (n)$.
3. Compute $\frac{n}{\phi(n)}$.
4. If larger than current max, store $n$ and $\frac{n}{\phi(n)}$ as the new maximum.

# Importing the required modules



In [0]:
import numpy as np
import scipy as sp
import collections as coll
from numba import jit
import time

# Prime Factors Formula

In [0]:

def Prime_Factors(n,returnType='counter'):
    """
    Obtains the prime factors for a number.

    Should be used only for non-prime numbers. Passing in a prime number will return an empty dict.

    Args:
    - n:int = Number to be prime factorized.

    - returnType:'counter' or 'list' = Determines the data type of the result returned. Default is 'counter', which\
will return a collections counter object. 

    Returns:
    - A collections Counter dict, which has the key-value pair of 'prime':'occurences'.

    """

    # Initializing the variables
    primes=Prime_Number_Generator()
    i=next(primes)
    result=[]
    _n=n # used for division

    # Checking the factors
    while i<=_n: # terminates when i>=n
        if (_n/i)%1==0: # perfect division
            _n=_n/i 
            result.append(i) # add to result list
            continue # continue without incrementing i.
        else:
            i=next(primes) # get the next prime candidate to try

    # Removing the original n value if present in list
    try:
        result.remove(n)
    except:
        pass
        
    # Returning the results
    if returnType=='counter':
        return coll.Counter(result)
    elif returnType=='list':
        return result


def Prime_Sieve(ul):
    """
    Args:
    - ul:int = Upper Limit of the prime sieve.

    Returns:
    - result = List of prime numbers up till the upper limit.
    """
    # Known conditions
    if ul<=1:
        return np.array([])
    elif ul==2:
        return np.array([2])
    elif ul==3:
        return np.array([2,3])

    # Forming the initial array
    ini=np.arange(3,ul,2) # creating the initial array with odd values only.
    result=ini.copy() # make a copy for the array

    # Looping through the array to eliminate non-primes
    i=0 # initial position value.
    while (ini[i])**2<max(result):
        x=ini[i]
        x2=x**2
        comp=np.arange(x2,max(result)+1,x) # comparison array
        result=np.setdiff1d(result,comp) # returns result that removes the common elements & sorted.
        i+=1

    # Adding 2 to the start of the array
    result=np.insert(result,0,2)

    return result

def Prime_Number_Generator():
    """
    Continually generate a list of prime numbers.

    Do not use this function if a list of primes must be generated. Use the Prime_Sieve function instead that is faster and more efficient.
    """
    # initial prime number
    i=2
    yield i
    primes=set()

    # following prime numbers
    primes.add(i)
    i+=1 # i==3
    while True:
        pFlag=all([(i/j)%1!=0 for j in primes]) # returns True if the current i cannot be divided by the past prime values.
        if pFlag:
            yield i
            primes.add(i)
        i+=1

def Euler_Toitent(n):
    """
    Determines the Euler Toitent value for the integer n.

    Args:

    - n:int = Number to determine the Euler Toitent function value.

    Returns:

    - prdt:int = Euler toitent value for number n.
    """
    primes=set(Prime_Factors(n,returnType='list'))

    if len(primes)==0: # indicates that it is a prime number
        return int(n-1)

    prdt=1
    for p in primes:
        prdt*=1-(1/p)
    prdt*=n
    return int(prdt)

# Alternate solution procedure

- After attempting to check all the non-prime numbers using the Euler_Toitent function, it was found that the algorithm was too slow to completely check all the values for $n \leq 1000000$.

- Therefore, the brute force method is not feasible. An alternative solution method must be developed.

## Mathematical solution

- Based on the above formula, it can be seen that the result for $\frac{n}{\phi(n)}$ can be written as follows:

\begin{equation}
\frac{n}{\phi(n)}
=\frac{n}{n \cdot \prod \left(1-\frac{1}{p_i}\right)}
=\frac{1}{\prod \left(1-\frac{1}{p_i} \right)}
\end{equation}

- $\therefore$ In order to maximize $\frac{n}{\phi(n)}$ , the number of $p_i$ should increase, and the value of each $p_i$ term should also be maximized.

- Also, it was noted that only unique $p_i$ terms were considered within the formula to compute $\frac{n}{\phi(n)}$. 
    - For instance, $n_6=(2,3)$ while $n_{12}=(2,2,3)$, 
    but both their $\frac{n}{\phi(n)}$ values are 3.
    - Therefore, in this case since the limit is $n\leq 1000000$, the smaller $n$ value with the same number of unique prime factors is preferential.

- In this case, the result for $\frac{n}{\phi(n)}$ is given 
as $\frac{n}{n-1}$. 

\begin{equation}
\therefore \lim_{n \to \infty} \frac{n}{n-1}=1 \\
\therefore \text{prime numbers should not be considered in the maximization of }
\frac{n}{\phi(n)}
\end{equation}



In [17]:
def Main():

    # CREATE THE PRIME GENERATION OBJECT
    primes=Prime_Number_Generator()

    # CREATE A LOOP THAT WILL BE TERMINATED WHEN N>1000000
    i=next(primes)
    n=1
    result=set()
    ul=1000000
    while True:
        result.add(i)
        n*=i
        ## TERMINATION CONDITION
        if n>ul:
            result.remove(i) # remove the previous multiple
            n*=1/i # divide by the previous multiple
            break # break out of the while loop
        i=next(primes)
    
    # RETURN THE RESULT #
    eT=1
    for i in result:
        eT*=1-(1/i) # denominator product
    eT=1/eT # inverse

    print(f"The maximum value of n/phi(n) is {int(n)}={eT}.")


if __name__=='__main__':
    start=time.time()
    Main()
    end=time.time()
    print(f"The solution was found in {end-start} seconds.")

The maximum value of n/phi(n) is 510510=5.539388020833332.
The solution was found in 0.00012683868408203125 seconds.


# Discussion

- Initially, the problem was attempted to be solved by generating all the prime divisors for each value and comparing them individually.
    - However, this was too computationally intensive, and thus another method had to be employed.

- Using the revised mathematical method, the solution was found in a reasonable amount of computational time.